# Homework and bake-off: Stanford Sentiment Treebank

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Methodological note](#Methodological-note)
1. [Set-up](#Set-up)
1. [A softmax baseline](#A-softmax-baseline)
1. [RNNClassifier wrapper](#RNNClassifier-wrapper)
1. [Error analysis](#Error-analysis)
1. [Homework questions](#Homework-questions)
  1. [Reproducing Socher et al.'s NaiveBayes baselines [2 points]](#Reproducing-Socher-et-al.'s-NaiveBayes-baselines-[2-points])
  1. [Sentiment words alone [2 points]](#Sentiment-words-alone-[2-points])
  1. [A more powerful vector-summing baseline [2 points]](#A-more-powerful-vector-summing-baseline-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to the Stanford Sentiment Treebank (SST). The homework questions ask you to implement some baseline systems, and the bake-off challenge is to define a system that does extremely well at the SST task.

We'll focus on the ternary task as defined by `sst.ternary_class_func`.

The SST test set will be used for the bake-off evaluation. This dataset is already publicly distributed, so we are counting on people not to cheat by develping their models on the test set. You must do all your development without using the test set at all, and then evaluate exactly once on the test set and turn in the results, with no further system tuning or additional runs. __Much of the scientific integrity of our field depends on people adhering to this honor code__. 

Our only additional restriction is that __you cannot make any use of the subtree labels__. This corresponds to the 'Root' condition in the paper. As we discussed in class, the subtree labels are a really interesting feature of SST, but bringing them in results in a substantially different learning problem.

One of our goals for this homework and bake-off is to encourage you to engage in __the basic development cycle for supervised models__, in which you

1. Write a new feature function. We recommend starting with something simple.
1. Use `sst.experiment` to evaluate your new feature function, with at least `fit_softmax_classifier`.
1. If you have time, compare your feature function with `unigrams_phi` using `sst.compare_models` or `sst.compare_models_mcnemar`. (For discussion, see [this notebook section](sst_02_hand_built_features.ipynb#Statistical-comparison-of-classifier-models).)
1. Return to step 1, or stop the cycle and conduct a more rigorous evaluation with hyperparameter tuning and assessment on the `dev` set.

[Error analysis](#Error-analysis) is one of the most important methods for steadily improving a system, as it facilitates a kind of human-powered hill-climbing on your ultimate objective. Often, it takes a careful human analyst just a few examples to spot a major pattern that can lead to a beneficial change to the feature representations.

## Methodological note

You don't have to use the experimental framework defined below (based on `sst`). However, if you don't use `sst.experiment` as below, then make sure you're training only on `train`, evaluating on `dev`, and that you report with 

```
from sklearn.metrics import classification_report
classification_report(y_dev, predictions)
```
where `y_dev = [y for tree, y in sst.dev_reader(class_func=sst.ternary_class_func)]`. We'll focus on the value at `macro avg` under `f1-score` in these reports.

## Set-up

See [the first notebook in this unit](sst_01_overview.ipynb#Set-up) for set-up instructions.

In [19]:
from collections import Counter
from nltk.tree import Tree
import numpy as np
import os
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
import sst
import torch.nn as nn
from torch_rnn_classifier import TorchRNNClassifier
from torch_tree_nn import TorchTreeNN
import utils

In [20]:
SST_HOME = os.path.join('data', 'trees')

## A softmax baseline

This example is here mainly as a reminder of how to use our experimental framework with linear models.

In [21]:
def unigrams_phi(tree):
    """The basis for a unigrams feature function.
    
    Parameters
    ----------
    tree : nltk.tree
        The tree to represent.
    
    Returns
    -------    
    Counter
        A map from strings to their counts in `tree`. (Counter maps a 
        list to a dict of counts of the elements in that list.)
    
    """
    return Counter(tree.leaves())

Thin wrapper around `LogisticRegression` for the sake of `sst.experiment`:

In [22]:
def fit_softmax_classifier(X, y):        
    mod = LogisticRegression(
        fit_intercept=True,
        solver='liblinear',
        multi_class='ovr')
    mod.fit(X, y)
    return mod

The experimental run with some notes:

In [23]:
softmax_experiment = sst.experiment(
    SST_HOME,
    unigrams_phi,                      # Free to write your own!
    fit_softmax_classifier,            # Free to write your own!
    train_reader=sst.train_reader,     # Fixed by the competition.
    assess_reader=sst.dev_reader,      # Fixed until the bake-off.
    class_func=sst.ternary_class_func) # Fixed by the bake-off rules.

              precision    recall  f1-score   support

    negative      0.628     0.689     0.657       428
     neutral      0.343     0.153     0.211       229
    positive      0.629     0.750     0.684       444

    accuracy                          0.602      1101
   macro avg      0.533     0.531     0.518      1101
weighted avg      0.569     0.602     0.575      1101



`softmax_experiment` contains a lot of information that you can use for analysis; see [this section below](#Error-analysis) for starter code.

## RNNClassifier wrapper

This section illustrates how to use `sst.experiment` with RNN and TreeNN models.

To featurize examples for an RNN, we just get the words in order, letting the model take care of mapping them into an embedding space.

In [25]:
def rnn_phi(tree):
    return tree.leaves()    

The model wrapper gets the vocabulary using `sst.get_vocab`. If you want to use pretrained word representations in here, then you can have `fit_rnn_classifier` build that space too; see [this notebook section for details](sst_03_neural_networks.ipynb#Pretrained-embeddings).

In [26]:
def fit_rnn_classifier(X, y):    
    sst_glove_vocab = utils.get_vocab(X, n_words=10000)     
    mod = TorchRNNClassifier(
        sst_glove_vocab, 
        eta=0.05,
        embedding=None,
        batch_size=1000,
        embed_dim=50,
        hidden_dim=50,
        max_iter=50,
        l2_strength=0.001,
        bidirectional=True,
        hidden_activation=nn.ReLU())
    mod.fit(X, y)
    return mod

In [27]:
rnn_experiment = sst.experiment(
    SST_HOME,
    rnn_phi,
    fit_rnn_classifier, 
    vectorize=False,  # For deep learning, use `vectorize=False`.
    assess_reader=sst.dev_reader)

Finished epoch 32 of 50; error is 2.7091598957777023

KeyboardInterrupt: 

## Error analysis

This section begins to build an error-analysis framework using the dicts returned by `sst.experiment`. These have the following structure:

```
'model': trained model
'train_dataset':
   'X': feature matrix
   'y': list of labels
   'vectorizer': DictVectorizer,
   'raw_examples': list of raw inputs, before featurizing   
'assess_dataset': same structure as the value of 'train_dataset'
'predictions': predictions on the assessment data
'metric': `score_func.__name__`, where `score_func` is an `sst.experiment` argument
'score': the `score_func` score on the assessment data
```
The following function just finds mistakes, and returns a `pd.DataFrame` for easy subsequent processing:

In [24]:
def find_errors(experiment):
    """Find mistaken predictions.
    
    Parameters
    ----------
    experiment : dict
        As returned by `sst.experiment`.
        
    Returns
    -------
    pd.DataFrame
    
    """
    raw_examples = experiment['assess_dataset']['raw_examples']
    raw_examples = [" ".join(tree.leaves()) for tree in raw_examples]
    df = pd.DataFrame({
        'raw_examples': raw_examples,
        'predicted': experiment['predictions'],
        'gold': experiment['assess_dataset']['y']})
    df['correct'] = df['predicted'] == df['gold']
    return df

In [ ]:
softmax_analysis = find_errors(softmax_experiment)

In [ ]:
rnn_analysis = find_errors(rnn_experiment)

Here we merge the sotmax and RNN experiments into a single DataFrame:

In [ ]:
analysis = softmax_analysis.merge(
    rnn_analysis, left_on='raw_examples', right_on='raw_examples')

analysis = analysis.drop('gold_y', axis=1).rename(columns={'gold_x': 'gold'})

The following code collects a specific subset of examples; small modifications to its structure will give you different interesting subsets:

In [ ]:
# Examples where the softmax model is correct, the RNN is not,
# and the gold label is 'positive'

error_group = analysis[
    (analysis['predicted_x'] == analysis['gold'])
    &
    (analysis['predicted_y'] != analysis['gold'])    
    &
    (analysis['gold'] == 'positive')
]

In [ ]:
error_group.shape[0]

In [ ]:
for ex in error_group['raw_examples'].sample(5):
    print("="*70)
    print(ex)

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Reproducing Socher et al.'s NaiveBayes baselines [2 points]

[Socher et al.](https://www.aclweb.org/anthology/D13-1170/) compare against (among other models) a NaiveBayes baseline with bigram features. See how close you can come to reproducing the performance of that model on the binary, root-only problem (values in the rightmost column of their Table 1, rows 1 and 3).

Specific tasks:

1. Write a bigrams feature function called `bigrams_phi` on the model of `unigrams_phi`. The included function `test_bigrams_phi` should help verify that you've done this correctly.
1. Write a function `fit_nb_classifier` that serves as a wrapper for `sklearn.naive_bayes.MultinomialNB`, which you can use with all its default arguments or change them as you see fit.
1. Use `sst.experiment` to run the experiments, assessing against `sst.dev_reader`.

Submit all the code you write for this, including any new import statements, and make sure your notebook embeds the output from running the code in step 3.

__A note on performance__: in our experience, the bigrams Naive Bayes model gets around 0.75. It's fine to submit answers with comparable numbers; the Socher et al. baselines are very strong. We're not evaluating how good your model is; we want to see your code, and we're interested to see what the range of F1 scores is across the whole class.

In [75]:
##### YOUR CODE HERE

def bigrams_phi(tree):
    tokens = ['<S>'] + tree.leaves() + ['</S>'] 
    #print(tokens)
    bigram = [(i,j) for i,j in zip(tokens[:-1],tokens[1:])]
    return Counter(bigram)

In [66]:
tree = Tree.fromstring("""(4 (2 NLU) (4 (2 is) (4 amazing)))""")
tree.leaves()
tokens = ['<S>'] + tree.leaves() + ['<S>'] 
print(tokens)
bigram = [(i,j) for i,j in zip(tokens[:-1],tokens[1:])]
print(Counter(bigram))

['<S>', 'NLU', 'is', 'amazing', '<S>']
Counter({('<S>', 'NLU'): 1, ('NLU', 'is'): 1, ('is', 'amazing'): 1, ('amazing', '<S>'): 1})


In [67]:
def test_bigrams_phi(func):
    """`func` should be `bigrams_phi`."""
    tree = Tree.fromstring("""(4 (2 NLU) (4 (2 is) (4 amazing)))""")
    result = bigrams_phi(tree)
    print(result)
    expected = {('<S>', 'NLU'): 1, 
                ('NLU', 'is'): 1, 
                ('is', 'amazing'): 1, 
                ('amazing', '</S>'): 1}
    assert result == expected, \
        "Expected {}\nGot {}".format(expected, result)

In [68]:
test_bigrams_phi(bigrams_phi)

['<S>', 'NLU', 'is', 'amazing', '</S>']
Counter({('<S>', 'NLU'): 1, ('NLU', 'is'): 1, ('is', 'amazing'): 1, ('amazing', '</S>'): 1})


In [77]:
from sklearn.naive_bayes import MultinomialNB

def fit_nb_classifier(X,y):
    nb_model = MultinomialNB() 
    nb_model.fit(X,y)
    return nb_model

In [78]:
nb_exp = sst.experiment(SST_HOME,
                        bigrams_phi,
                        fit_nb_classifier,
                        class_func=sst.ternary_class_func, 
                        train_reader = sst.train_reader,
                        assess_reader = sst.dev_reader)

              precision    recall  f1-score   support

    negative      0.598     0.678     0.635       428
     neutral      0.235     0.052     0.086       229
    positive      0.600     0.764     0.672       444

    accuracy                          0.582      1101
   macro avg      0.478     0.498     0.464      1101
weighted avg      0.523     0.582     0.536      1101



### Sentiment words alone [2 points]

NLTK includes an easy interface to [Minqing Hu and Bing Liu's __Opinion Lexicon__](https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html), which consists of a list of positive words and a list of negative words. How much of the ternary SST story does this lexicon tell?

For this problem, submit code to do the following:

1. Create a feature function `op_unigrams` on the model of `unigrams_phi` above, but filtering the vocabulary to just items that are members of the Opinion Lexicon. Submit this feature function.

1. Evaluate your feature function with `sst.experiment`, with all the same parameters as were used to create `softmax_experiment` in [A softmax baseline](#A-softmax-baseline) above, except of course for the feature function.

1. Use `utils.mcnemar` to compare your feature function with the results in `softmax_experiment`. The information you need for this is in `softmax_experiment` and your own `sst.experiment` results. Submit your evaluation code. You can assume `softmax_experiment` is already in memory, but your code should create the other objects necessary for this comparison.

In [86]:
from nltk.corpus import opinion_lexicon

# Use set for fast membership checking:
positive = set(opinion_lexicon.positive())
negative = set(opinion_lexicon.negative())
tree = Tree.fromstring("""(4 (2 terrible) (4 (2 is) (4 amazing)))""")
tree.leaves()

##### YOUR CODE HERE

def op_unigrams(tree):
    tokens = []

    for word in tree.leaves():
        if word in positive or word in negative:
            tokens.append(word)

    print(tokens)

    return Counter(tokens)


In [87]:
softmax_experiment = sst.experiment(
    SST_HOME,
    op_unigrams,                      # Free to write your own!
    fit_softmax_classifier,            # Free to write your own!
    train_reader=sst.train_reader,     # Fixed by the competition.
    assess_reader=sst.dev_reader,      # Fixed until the bake-off.
    class_func=sst.ternary_class_func) # Fixed by the bake-off rules.

[]
['gorgeously']
['intrusive']
['enough', 'gold']
['charming']
['fascinating', 'playful']
['richness', 'madness', 'astonishing']
['charm', 'humour']
['ingeniously']
['exceeds']
['fun', 'good', 'good', 'good', 'good', 'good']
['suffering', 'unaffected']
['like']
['quiet', 'pure', 'shortage', 'respectable']
['fun', 'excellent']
['terrific', 'innovation']
['disapproval']
[]
[]
['love']
['love']
['well']
[]
['terrific']
['fans']
[]
['fantastic']
['worth']
['work']
[]
['passion']
['nice']
['vulgar', 'liked']
['lacks']
['silly', 'good']
['weaknesses', 'creative', 'interference']
['charm', 'lies', 'cuteness']
[]
['died']
['humor', 'fluent', 'witty', 'cute']
[]
['brave', 'lead']
['goodwill', 'slow']
['works', 'unusual']
[]
['sour', 'emptiness', 'relentless', 'gaiety']
['like', 'great', 'fun']
['warning']
['famous', 'critic']
['beautiful', 'evocative', 'works']
['appealing', 'work']
['great', 'pleasure']
['shameless', 'promised']
['wanton', 'amiable', 'great']
['like', 'killer', 'better']
['pr

['boredom']
['paranoid', 'unsettling']
['afraid']
['detract', 'affection', 'favorite']
['problem', 'shortness', 'disappoints']
['ardent', 'fans', 'hopelessly']
['plot', 'twists', 'attraction', 'desperately']
['brilliant', 'brutal', 'crime', 'drips']
['denial', 'striking', 'sadness']
['nice']
['proving']
[]
['convoluted', 'worth']
['amazing']
['win']
['courageous', 'remarkable', 'memorable']
['powerful']
['achievement']
[]
['important', 'talent']
['keen', 'soulful', 'grieving']
['engaging', 'diatribes', 'grand', 'passion']
['sumptuous', 'wildly']
[]
['refreshing', 'like']
['patient', 'uncompromising', 'abuse']
['outstanding']
['ingenious', 'damaged', 'despair']
['best']
['lie']
['surreal']
['vivid', 'convincing', 'fine', 'rapid']
[]
['unexpectedly', 'sweet']
['challenging']
['humorous', 'illuminating']
[]
['modern', 'fails', 'fascinate']
['impeccable']
['entertaining']
[]
['like']
[]
['creepy', 'powerful', 'self-destructive', 'unsettling']
[]
['best']
['top-notch']
['superb', 'magnifice

['recommend']
['delightful', 'witty', 'improbable', 'romantic', 'zippy', 'fun']
['tops', 'leads']
['good']
['savage', 'humor', 'falling']
['divine']
['impair']
['interesting', 'conspiracy', 'worthwhile']
[]
['issue', 'depression']
['shame', 'marvelous']
['sloppy', 'brilliance', 'sparkling', 'shame']
['good']
[]
['fine']
['reluctant', 'incapable']
['scariest']
[]
['better', 'magical', 'fun']
['modesty', 'predicament']
['classic', 'remarkably', 'work']
['cool', 'intriguingly']
['well', 'conflict', 'like']
['rude', 'holy', 'fool', 'cutthroat']
['subversive', 'unrelenting', 'bleak', 'work', 'sad']
['frighten', 'disturb', 'works', 'spectacularly', 'well']
['playful', 'openness']
['uncompromising']
['pleasant', 'spite', 'joy', 'first-rate']
['invisible']
['right']
['fascinating', 'mystery', 'controversy']
[]
['dashing', 'inventive']
['fine', 'great']
['flat-out', 'amusing', 'endearing', 'fabulous', 'solid', 'noteworthy', 'delicious', 'effective']
['remarkable']
['clears', 'cynicism', 'right'

['strong']
['love', 'robust']
[]
[]
['beautiful', 'aching', 'sadness']
[]
[]
['suffers', 'melodramatic']
['ignore']
['funny', 'heartwarming', 'well', 'enjoyable']
['debts', 'entertaining']
['earnest', 'workable', 'terrific']
[]
['loved']
['trumpet']
['calm', 'thoughtful', 'failure']
['appealing', 'appalling', 'creepy', 'love']
['originality', 'humour']
['spooky']
['challenging', 'cohesive']
['effectiveness']
['gangster', 'fair', 'intriguing', 'bizarre']
['incredible']
['exhausted', 'remarkable', 'shockingly', 'little-known']
[]
['witty', 'appealing', 'enough', 'good']
['skillful']
['winning']
['delighted', 'fast', 'funny']
[]
['unsettling', 'memorable', 'proud']
['appealing', 'bright', 'fans']
['mesmerizing', 'weep']
['strongest']
['masterpiece', 'won', 'wins']
['modern']
[]
['enjoyed']
['inspirational']
['deft', 'delightful', 'triumph']
['taut', 'feisty', 'subversive']
['fair']
['fairly', 'enjoyable']
[]
['work']
['entertaining', 'fall', 'grace', 'gratify', 'overrun', 'corrupt', 'hedo

['genial', 'buoyant', 'sporadic']
['fanciful', 'love', 'humility']
['privileged']
['dignity', 'perfect', 'proud']
['uncompromising']
['loves', 'trust']
['ideal']
['love', 'love']
['treasure']
['well', 'perfectly']
['right']
['gracefully']
['funny', 'bad']
['intolerable', 'well', 'worth']
['gem', 'favored']
['defiantly', 'daring']
['hilarious', 'tedious']
['break', 'sensitive', 'delicate']
['perfect', 'worth']
['favor', 'outrageousness']
['dark', 'gritty', 'funny', 'gem']
['supporting']
['engrossing']
['classic']
['durable']
['invaluable', 'extraordinary', 'charm', 'readily']
[]
['melodramatic', 'enough']
['masterpiece']
['honest', 'appeal', 'willing', 'succumb']
['appreciate', 'wonderful']
['farce', 'appeal']
['limits']
['farcical', 'grim', 'good']
['compassionate']
['annoyances', 'capable', 'great']
[]
['comfort']
['modest', 'aspirations']
['funny', 'creative']
['worth']
[]
['cheesy', 'cool', 'smart', 'cleverly', 'nicely']
['fateful']
['remarkable']
['terrific', 'like', 'villains']
['

...

['clear', 'doubt', 'interesting']
['falls', 'sloppy', 'insultingly', 'unbelievable', 'crazy', 'interesting']
['disaster', 'ridiculous']
['loud', 'goofy']
['greasy']
['excruciatingly', 'tedious', 'torture', 'appealing']
['intriguing', 'distasteful', 'creepy']
['boredom']
['like', 'salute', 'cheesy']
['hard', 'mess']
['handsome', 'amusing', 'levity', 'bleak', 'mediocrity']
['excellent', 'unfortunately']
[]
['entertaining', 'hard', 'sucker', 'impossible']
['ambitious', 'eager', 'prolific', 'plot', 'comfortably']
['mystery', 'rich', 'first-rate', 'talent']
['odd']
['interesting']
[]
['limitations', 'clear']
['dull', 'death', 'plot']
[]
['poorly', 'coherent', 'believable', 'zeal', 'propaganda']
['interesting', 'conflict', 'lacks', 'fails', 'struggle', 'meaningful']
['right', 'right']
['sincere', 'fault', 'unfortunately', 'fun']
['distinctive', 'lead']
['classic']
['free']
['good']
['slow-moving', 'joy', 'preferring', 'humiliation']
['limp']
['plot', 'maddeningly', 'repetitive', 'scream']
['

['sorry', 'explode']
['loses']
['unclean', 'liked']
[]
['manipulative', 'bland']
['pretty', 'like', 'coherence']
['stuck']
['right', 'difficult', 'cuss', 'bungling']
['loud', 'tired']
['relentlessly', 'oppositions']
['pathetic', 'exploitation', 'sincere', 'worse']
['trickery', 'clever', 'flashy', 'silly', 'overkill']
['charm']
['like', 'work']
['badly', 'dark', 'humor', 'poetic', 'tragedy']
['fool']
['ambitious', 'wrong']
['exciting']
['dark', 'cartoonish', 'well']
['like', 'generous']
['convincing']
[]
['strained', 'fun']
[]
['work', 'works', 'well', 'right', 'wrong']
['hard']
[]
['shallow', 'immature']
['brisk', 'excellent']
['better']
['blow', 'decent']
['bad']
['fanatic', 'best']
['cautionary', 'superficiality', 'superficial']
[]
['magic', 'stronger']
['graceful', 'clunky']
['funny', 'miss']
['stupid', 'goofy', 'cute']
['classic']
['undone', 'enough', 'alienate', 'cliched', 'hardened']
['coherent']
['sad', 'drag', 'shockingly', 'worse']
[]
['hate', 'lacks', 'skill']
['like']
['vagu

['jarring']
['interesting', 'faithful']
['endearing', 'humorous', 'spooky', 'bland']
[]
['bleed', 'humor', 'fun']
['well', 'flawed', 'dazzling']
[]
['lost', 'best']
['interesting']
['charm', 'fun']
['unlikely', 'appreciated']
['like', 'futile', 'hastily', 'crashing']
[]
[]
['manipulative', 'empowerment', 'abuse']
['exceedingly', 'pleasant', 'offend']
['easy']
['profoundly', 'stupid']
['awards', 'phony', 'humility', 'grotesque']
['lazy']
['plot']
['enough', 'doldrums']
['work', 'love', 'support']
['impossible', 'plot']
['hottest', 'strangely', 'enough', 'swagger', 'shameless']
[]
['fascinating', 'good']
[]
['good', 'good', 'despairingly', 'awful']
['clamorous']
['depressed']
['terrible', 'delicate', 'farcical', 'loathsome']
['wrong', 'unbearable']
['inexperienced']
['limp']
['repetitive', 'ragged', 'horrifying']
[]
['stiff', 'blow']
[]
['guilty', 'pleasure', 'incredibly', 'inane', 'enjoyable']
['relentlessly', 'like']
['entertaining', 'enough']
[]
['irksome', 'tiresome', 'utterly', 'sat

['humor']
[]
[]
['sucked', 'monster', 'silly', 'scary']
['slow']
['accuse', 'bad']
['prefer']
[]
['nice', 'like']
['interesting']
['conspiracy', 'best']
['uncomfortably', 'losing']
['rejected', 'boring', 'crap']
['fatally', 'consistent', 'lacks']
['weak']
['joke']
['joke']
['joke', 'annoying']
['fairly', 'weak']
['lousy', 'lead']
['hilarious', 'fine', 'shame', 'mess']
['worst', 'suffering']
['dumb']
['heartfelt']
['sugar-coated', 'valuable']
['wrong']
['cheapen']
['realistic']
['unlikely', 'inspire', 'savvy']
['right']
['cynicism', 'ugly', 'shabby', 'muddy']
['good', 'enough', 'bad', 'enough', 'repulse', 'fans']
['silly', 'silly']
['hopelessly']
['menace']
['like']
['funny', 'charming']
[]
['frightful', 'vanity', 'doubt', 'debt']
['muddy', 'rife']
['good']
['enough']
['listless', 'futuristic', 'memorable']
[]
['laughably', 'unbearable', 'offensive']
['sadly', 'proving', 'talent']
['mess', 'pathetic', 'innovative', 'realistic']
['pathetic']
['suffers', 'enough', 'merit']
['bland', 'utte

['unnecessary']
['dreary', 'indulgence']
['crushing', 'disappointment']
[]
['waste']
['confusing']
[]
[]
['fresh']
['lame']
[]
['amusing']
[]
['entertaining']
[]
[]
['confusing']
[]
['missed']
[]
[]
['naive']
[]
['lame']
[]
['poorly']
['horrible']

['memorable', 'disturbing', 'unusual', 'successful']
['exhilarating', 'futuristic', 'twists', 'best']
['spooky', 'creepy']
['enjoy', 'fast-paced', 'envious']
['admirable', 'hypocrisy', 'uneven', 'humor', 'tragedy']
['hard', 'humor', 'right']
['perfectly', 'well']
['heartwarming']
['perfect']
[]
['dissidents', 'alarmingly']
['crack']
['betrayals', 'celebrated']
['corruption', 'ruthlessness']
['pretty', 'pleasant', 'enough']
['delicious', 'grand']
['plot', 'uncomplicated', 'morality', 'well', 'affable']
['quiet', 'threatening', 'unforgettable']
['illuminating']
['funny', 'fast', 'loose', 'accessible']
['intelligent', 'difficult']
[]
['concerned']
['great']
['entertaining', 'colorful', 'crime', 'intimate']
['tidy']
[]
['earnest', 'illusion']
['

### A more powerful vector-summing baseline [2 points]

In [Distributed representations as features](sst_03_neural_networks.ipynb#Distributed-representations-as-features), we looked at a baseline for the ternary SST problem in which each example is modeled as the sum of its 50-dimensional GloVe representations. A `LogisticRegression` model was used for prediction. A neural network might do better with these representations, since there might be complex relationships between the input feature dimensions that a linear classifier can't learn. 

To address this question, rerun the experiment with `TorchShallowNeuralClassifier` as the classifier. Specs:
* Use `sst.experiment` to conduct the experiment. 
* Using 3-fold cross-validation, exhaustively explore this set of hyperparameter combinations:
  * The hidden dimensionality at 50, 100, and 200.
  * The hidden activation function as `nn.Tanh` or `nn.ReLU`.
* (For all other parameters to `TorchShallowNeuralClassifier`, use the defaults.)

For this problem, submit code to do the following:

1. Your model wrapper function around `TorchShallowNeuralClassifier`. This function should implement the requisite cross-validation; see [this notebook section](sst_02_hand_built_features.ipynb#Hyperparameter-search) for examples.
1. The classification report as printed by `sst.experiment`. (This will print out when you run `sst.experiment`. That print-out suffices.)
2. The optimal hyperparameters chosen in your experiment. (This too will print out when you run `sst.experiment`. The print-out again suffices.)

We're not evaluating the quality of your model. (We've specified the protocols completely, but there will still be variation in the results.) However, the primary goal of this question is to get you thinking more about this strikingly good baseline feature representation scheme for SST, so we're sort of hoping you feel compelled to try out variations on your own.

In [5]:
DATE_HOME = 'data'

GLOVE_HOME = os.path.join(DATE_HOME, 'glove.6B')

VSMDATA_HOME = os.path.join(DATE_HOME, 'vsmdata')

SST_HOME = os.path.join(DATE_HOME, 'trees')

In [94]:
##### YOUR CODE HERE
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
def torch_NN_classifier(X,y):
    mod = TorchShallowNeuralClassifier(hidden_dim=100, max_iter= 50)
    mod.fit(X,y)
    return mod

In [15]:
glove_lookup = utils.glove2dict(os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [96]:
def vsm_leaves_phi(tree, lookup, np_func=np.sum):
    """Represent `tree` as a combination of the vector of its words.
    
    Parameters
    ----------
    tree : nltk.Tree   
    lookup : dict
        From words to vectors.
    np_func : function (default: np.sum)
        A numpy matrix operation that can be applied columnwise, 
        like `np.mean`, `np.sum`, or `np.prod`. The requirement is that 
        the function take `axis=0` as one of its arguments (to ensure
        columnwise combination) and that it return a vector of a 
        fixed length, no matter what the size of the tree is.
    
    Returns
    -------
    np.array, dimension `X.shape[1]`
            
    """      
    allvecs = np.array([lookup[w] for w in tree.leaves() if w in lookup])    
    if len(allvecs) == 0:
        dim = len(next(iter(lookup.values())))
        feats = np.zeros(dim)
    else:       
        feats = np_func(allvecs, axis=0)      
    return feats

In [97]:
def glove_leaves_phi(tree, np_func=np.sum):
    return vsm_leaves_phi(tree, glove_lookup, np_func=np_func)

In [99]:
nn_experiment = sst.experiment(
    SST_HOME,
    glove_leaves_phi,                      # Free to write your own!
    torch_NN_classifier,            # Free to write your own!
    train_reader=sst.train_reader,     # Fixed by the competition.
    assess_reader=sst.dev_reader,      # Fixed until the bake-off.
    class_func=sst.ternary_class_func,
    vectorize=False) # Fixed by the bake-off rules.

Finished epoch 50 of 50; error is 5.5171793103218085

              precision    recall  f1-score   support

    negative      0.610     0.680     0.643       428
     neutral      0.360     0.197     0.254       229
    positive      0.641     0.721     0.679       444

    accuracy                          0.596      1101
   macro avg      0.537     0.532     0.525      1101
weighted avg      0.571     0.596     0.577      1101



In [111]:
def fit_nn_with_crossvalidation(X, y):

    basemod = TorchShallowNeuralClassifier()
    cv = 3
    param_grid = {'hidden_dim': [50,10,200], 'hidden_activation': [nn.Tanh(),nn.ReLU()]}    
    best_mod = utils.fit_classifier_with_crossvalidation(
        X, y, basemod, cv, param_grid)
    return best_mod

In [112]:
nn_experiment = sst.experiment(
    SST_HOME,
    glove_leaves_phi,                      # Free to write your own!
    fit_nn_with_crossvalidation,            # Free to write your own!
    train_reader=sst.train_reader,     # Fixed by the competition.
    assess_reader=sst.dev_reader,      # Fixed until the bake-off.
    class_func=sst.ternary_class_func,
    vectorize=False) # Fixed by the bake-off rules.

Finished epoch 100 of 100; error is 3.1797873973846436

Best params: {'hidden_activation': Tanh(), 'hidden_dim': 200}
Best score: 0.422
              precision    recall  f1-score   support

    negative      0.635     0.561     0.596       428
     neutral      0.298     0.210     0.246       229
    positive      0.609     0.770     0.680       444

    accuracy                          0.572      1101
   macro avg      0.514     0.514     0.507      1101
weighted avg      0.554     0.572     0.557      1101



In [114]:
print(nn_experiment['model'])
print(nn_experiment['metric'])
print(nn_experiment['score'])

TorchShallowNeuralClassifier(
	hidden_dim=200,
	hidden_activation=Tanh(),
	batch_size=1028,
	max_iter=100,
	eta=0.01,
	optimizer=<class 'torch.optim.adam.Adam'>,
	l2_strength=0)
safe_macro_f1
0.5072026073501094


## McNemar's Test

In [115]:
m = utils.mcnemar(
    softmax_experiment['assess_dataset']['y'], 
    softmax_experiment['predictions'],
    nn_experiment['predictions'])

In [116]:
p = "p < 0.0001" if m[1] < 0.0001 else m[1]
print("McNemar's test: {0:0.02f} ({1:})".format(m[0], p))

McNemar's test: 0.07 (0.7938193831730679)


### Your original system [3 points]

Your task is to develop an original model for the SST ternary problem, using only the root-level labels (again, __you cannot make any use of the subtree labels__). There are many options. If you spend more than a few hours on this homework problem, you should consider letting it grow into your final project! Here are some relatively manageable ideas that you might try:

1. We didn't systematically evaluate the `bidirectional` option to the `TorchRNNClassifier`. Similarly, that model could be tweaked to allow multiple LSTM layers (at present there is only one), and you could try adding layers to the classifier portion of the model as well.

1. We've already glimpsed the power of rich initial word representations, and later in the course we'll see that smart initialization usually leads to a performance gain in NLP, so you could perhaps achieve a winning entry with a simple model that starts in a great place.

1. The [practical introduction to contextual word representations](contextualreps.ipynb) (to be discussed later in the quarter) covers pretrained representations and interfaces that are likely to boost the performance of any system.

1. The `TreeNN` and `TorchTreeNN` don't perform all that well, and this could be for the same reason that RNNs don't peform well: the gradient signal doesn't propagate reliably down inside very deep trees. [Tai et al. 2015](https://aclanthology.info/papers/P15-1150/p15-1150) sought to address this with TreeLSTMs, which are fairly easy to implement in PyTorch.

1. In the [distributed representations as features](#Distributed-representations-as-features) section, we just summed  all of the leaf-node GloVe vectors to obtain a fixed-dimensional representation for all sentences. This ignores all of the tree structure. See if you can do better by paying attention to the binary tree structure: write a function `glove_subtree_phi` that obtains a vector representation for each subtree by combining the vectors of its daughters, with the leaf nodes again given by GloVe (any dimension you like) and the full representation of the sentence given by the final vector obtained by this recursive process. You can decide on how you combine the vectors. 

1. If you have a lot of computing resources, then you can fire off a large hyperparameter search over many parameter values. All the model classes for this course are compatible with the `scikit-learn` and [scikit-optimize](https://scikit-optimize.github.io) methods, because they define the required functions for getting and setting parameters.

We want to emphasize that this needs to be an __original__ system. It doesn't suffice to download code from the Web, retrain, and submit. You can build on others' code, but you have to do something new and meaningful with it.

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [6]:
# Enter your system description in this cell.
# Please do not remove this comment.
X_rnn_train, y_rnn_train = sst.build_rnn_dataset(
    SST_HOME, sst.train_reader, class_func=sst.ternary_class_func)
X_rnn_dev, y_rnn_dev = sst.build_rnn_dataset(
    SST_HOME, sst.dev_reader, class_func=sst.ternary_class_func)


In [5]:
sst_full_train_vocab = utils.get_vocab(X_rnn_train)

In [6]:
print("sst_full_train_vocab has {:,} items".format(len(sst_full_train_vocab)))

sst_full_train_vocab has 18,279 items


In [7]:
sst_train_vocab = utils.get_vocab(X_rnn_train, n_words=10000)

In [8]:
sst_train_vocab[120]

'Able'

In [9]:
torch_rnn = TorchRNNClassifier(sst_train_vocab,embed_dim=50,hidden_dim=50,max_iter=25,eta=0.05)

In [10]:
%time _ = torch_rnn.fit(X_rnn_train, y_rnn_train)

Finished epoch 25 of 25; error is 0.9320394396781921

Wall time: 4min 18s


In [11]:
torch_rnn_dev_predictions = torch_rnn.predict(X_rnn_dev)

In [44]:
from sklearn.metrics import classification_report
print(classification_report(y_rnn_dev, torch_rnn_dev_predictions))

NameError: name 'torch_rnn_dev_predictions' is not defined

## Pretrained Glove Embeddings with RNN Classifier

In [16]:
glove_embedding, sst_glove_vocab = utils.create_pretrained_embedding(glove_lookup, sst_full_train_vocab)

In [17]:
glove_embedding.shape

(12785, 300)

In [18]:
torch_rnn_glove = TorchRNNClassifier(
    sst_glove_vocab,
    embedding=glove_embedding,
    hidden_dim=200,
    max_iter=25,
    eta=0.001) 

In [19]:
%time _ = torch_rnn_glove.fit(X_rnn_train, y_rnn_train)

Finished epoch 25 of 25; error is 4.992994189262396

Wall time: 15min 35s


In [20]:
torch_rnn_glove_dev_predictions = torch_rnn_glove.predict(X_rnn_dev)

In [21]:
print(classification_report(y_rnn_dev, torch_rnn_glove_dev_predictions))

              precision    recall  f1-score   support

    negative       0.69      0.70      0.69       428
     neutral       0.36      0.19      0.25       229
    positive       0.65      0.80      0.72       444

    accuracy                           0.63      1101
   macro avg       0.57      0.56      0.55      1101
weighted avg       0.60      0.63      0.61      1101



## TF-IDF + SGD Classifier

In [28]:
# Enter your system description in this cell.
# Please do not remove this comment.
X_rnn_train, y_rnn_train = sst.build_rnn_dataset(
    SST_HOME, sst.train_reader, class_func=sst.ternary_class_func)
X_rnn_dev, y_rnn_dev = sst.build_rnn_dataset(
    SST_HOME, sst.dev_reader, class_func=sst.ternary_class_func)

In [29]:
Counter(y_rnn_train)

Counter({'positive': 3610, 'neutral': 1624, 'negative': 3310})

In [30]:
Counter(y_rnn_dev)

Counter({'positive': 444, 'neutral': 229, 'negative': 428})

In [31]:
import re
import string
X_train_tfidf = []
regex = re.compile('[^a-zA-Z]')
for line in X_rnn_train:
    line_p = []
    for word in line:  
        word = word.translate ({ord(c): " " for c in string.punctuation})
        word = word.translate ({ord(c): " " for c in "_"})
        if(' ' in word):
            for sub_word in word.split():
                sub_word = regex.sub('', sub_word)
                if(len(sub_word)>1 and len(sub_word)<=14):
                    line_p.append(sub_word.lower())
        else:
            word = regex.sub('', word)
            if(len(word)>1 and len(word)<=14):
                line_p.append(word.lower())
    X_train_tfidf.append(" ".join(line_p))
print(len(X_train_tfidf))

8544


In [32]:
print(X_rnn_train[1])
print(X_train_tfidf[1])

['The', 'gorgeously', 'elaborate', 'continuation', 'of', '``', 'The', 'Lord', 'of', 'the', 'Rings', "''", 'trilogy', 'is', 'so', 'huge', 'that', 'a', 'column', 'of', 'words', 'can', 'not', 'adequately', 'describe', 'co-writer\\/director', 'Peter', 'Jackson', "'s", 'expanded', 'vision', 'of', 'J.R.R.', 'Tolkien', "'s", 'Middle-earth', '.']
the gorgeously elaborate continuation of the lord of the rings trilogy is so huge that column of words can not adequately describe co writer director peter jackson expanded vision of tolkien middle earth


In [33]:
import re
X_dev_tfidf = []
regex = re.compile('[^a-zA-Z]')
for line in X_rnn_dev:
    line_p = []
    for word in line:  
        word = word.translate ({ord(c): " " for c in string.punctuation})
        word = word.translate ({ord(c): " " for c in "_"})
        if(' ' in word):
            for sub_word in word.split():
                sub_word = regex.sub('', sub_word)
                if(len(sub_word)>1 and len(sub_word)<=14):
                    line_p.append(sub_word.lower())
        else:
            word = regex.sub('', word)
            if(len(word)>1 and len(word)<=14):
                line_p.append(word.lower())
    X_dev_tfidf.append(" ".join(line_p))
print(len(X_dev_tfidf))

1101


In [34]:
print(len(X_dev_tfidf))
print(X_dev_tfidf[1])

1101
no one goes unindicted here which is probably for the best


In [35]:
pd.DataFrame({'text':X_train_tfidf,'label':y_rnn_train}).to_csv("train_sst.csv")

In [36]:
pd.DataFrame({'text':X_dev_tfidf,'label':y_rnn_dev}).to_csv("dev_sst.csv")

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(lowercase=True, analyzer='word',max_df=1.0, ngram_range=(1,3), stop_words='english')
X_train_vect_norm = tfidf.fit_transform(X_train_tfidf)
print(X_train_vect_norm.shape)
features = tfidf.get_feature_names()
print("Total Features from Tf-Idf: ",len(features))

(8544, 131371)
Total Features from Tf-Idf:  131371


In [38]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
sgd1 = LinearSVC(class_weight='balanced')
clf = sgd1

In [39]:
clf.fit(X_train_vect_norm, y_rnn_train)

LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [40]:
X_dev_norm = tfidf.transform(X_dev_tfidf)
predictions = clf.predict(X_dev_norm)

In [41]:
Counter(predictions)

Counter({'positive': 557, 'negative': 457, 'neutral': 87})

In [42]:
Counter(y_rnn_dev)

Counter({'positive': 444, 'neutral': 229, 'negative': 428})

In [45]:
print(classification_report(y_rnn_dev, predictions))

              precision    recall  f1-score   support

    negative       0.65      0.69      0.67       428
     neutral       0.34      0.13      0.19       229
    positive       0.62      0.78      0.69       444

    accuracy                           0.61      1101
   macro avg       0.54      0.54      0.52      1101
weighted avg       0.58      0.61      0.58      1101



## Bake-off [1 point]

As we said above, the bake-off evaluation data is the official SST test-set release. For this bake-off, you'll evaluate your original system from the above homework problem on the test set, using the ternary class problem. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.
1. As noted above, __you cannot make any use of the subtree labels__.

The cells below this one constitute your bake-off entry.

Systems that enter will receive the additional homework point, and systems that achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [46]:
# Enter your bake-off assessment code in this cell. 
# Please do not remove this comment.

##### YOUR CODE HERE

X_rnn_test, y_rnn_test = sst.build_rnn_dataset(
    SST_HOME, sst.test_reader, class_func=sst.ternary_class_func)
#torch_rnn_glove_test_predictions = torch_rnn_glove.predict(y_rnn_test)
#print(classification_report(y_rnn_dev, torch_rnn_glove_test_predictions))

import re
X_test_tfidf = []
regex = re.compile('[^a-zA-Z]')
for line in X_rnn_test:
    line_p = []
    for word in line:  
        word = word.translate ({ord(c): " " for c in string.punctuation})
        word = word.translate ({ord(c): " " for c in "_"})
        if(' ' in word):
            for sub_word in word.split():
                sub_word = regex.sub('', sub_word)
                if(len(sub_word)>1 and len(sub_word)<=14):
                    line_p.append(sub_word.lower())
        else:
            word = regex.sub('', word)
            if(len(word)>1 and len(word)<=14):
                line_p.append(word.lower())
    X_test_tfidf.append(" ".join(line_p))
print(len(X_test_tfidf))
pd.DataFrame({'text':X_test_tfidf,'label':y_rnn_test}).to_csv("test_sst.csv")

X_test_norm = tfidf.transform(X_test_tfidf)
predictions = clf.predict(X_test_norm)

print(classification_report(y_rnn_test, predictions))

2210
              precision    recall  f1-score   support

    negative       0.68      0.69      0.69       912
     neutral       0.23      0.11      0.14       389
    positive       0.66      0.80      0.73       909

    accuracy                           0.63      2210
   macro avg       0.52      0.53      0.52      2210
weighted avg       0.59      0.63      0.61      2210



In [116]:
# On an otherwise blank line in this cell, please enter
# your macro-average F1 value as reported by the code above. 
# Please enter only a number between 0 and 1 inclusive.
# Please do not remove this comment.


!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
